<a href="https://colab.research.google.com/github/Haseebasif7/Fake-News-Prediction-SVM-/blob/main/Fake_News_Prediction(SVM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
df_train=pd.read_csv('/content/train.csv')
df_test=pd.read_csv('/content/test.csv')
df_submit=pd.read_csv('/content/submit.csv')

In [3]:
print(df_train.columns)
print('\n')
print(df_test.columns)

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')


Index(['id', 'title', 'author', 'text'], dtype='object')


In [4]:
print(df_train.shape,df_test.shape)

(20800, 5) (5200, 4)


In [5]:
print(df_train.info())
print('\n')
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5200 non-null   int64 
 1   title   5078 non-null   object
 2   author  4697 non-null   object
 3   text    5193 non-null   object
dtypes: int64(1), object(3)
memory usage: 162.6+ KB
None


In [6]:
df_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df_train1=df_train.drop(columns=['id','label'],axis=1) #Dropping the label and id column in train dataset
df_test1=df_test.drop(columns=['id'],axis=1) # Dropping the id columns in test dataset

In [8]:
df_train1['Content']=df_train1['author']+' '+df_train1['title']
df_test1['Content']=df_test1['author']+' '+df_test1['title']
df_train1.head() # making a column Content which contain both author and title and we will predict from it

,title,author,text,Content
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...


In [9]:
X_train=df_train1['Content'] #seperating Content from training set
Y_train=df_train.iloc[:,-1] # seperating label
X_test=df_test1['Content'] # seperating Content from test set
print(X_train.shape,Y_train.shape,X_test.shape)

(20800,) (20800,) (5200,)


In [10]:
X_train.fillna('',inplace=True)
X_test.fillna('',inplace=True) # Fill NAN with null strings

In [11]:
nltk.download('stopwords') # words that dont have importance in text
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
port_stem=PorterStemmer() # making object of Porter Stemmer used for stemming (finding root words for word)

In [13]:
def stemming(text): # stemming
    text=re.sub('[^a-zA-Z ]','',text)
    text=text.lower()
    text=text.split()
    stemmed_words=[]
    for word in text:
        if word not in stopwords.words('english'):
            stemmed_words.append(port_stem.stem(word))

    return ' '.join(stemmed_words)

In [14]:
X_train=X_train.apply(stemming)
X_test=X_test.apply(stemming)

In [15]:
print(X_train.head())

0    darrel lucu hous dem aid didnt even see comey ...
1    daniel j flynn flynn hillari clinton big woman...
2               consortiumnewscom truth might get fire
3    jessica purkiss civilian kill singl us airstri...
4    howard portnoy iranian woman jail fiction unpu...
Name: Content, dtype: object


In [16]:
Vectorizer=TfidfVectorizer() # changing text to numerical through tf-idf vectorizer
X_train=Vectorizer.fit_transform(X_train)
X_test=Vectorizer.transform(X_test)

In [17]:
print(X_train)

  (0, 15778)	0.27509776674930964
  (0, 2534)	0.3568969629559933
  (0, 7811)	0.23576831817995195
  (0, 8701)	0.2807585463060049
  (0, 3005)	0.23677503959610688
  (0, 13559)	0.24751839165094688
  (0, 4992)	0.2213068725389418
  (0, 4035)	0.2855963683829043
  (0, 262)	0.25988649489467747
  (0, 3833)	0.25725947050905124
  (0, 7142)	0.2101591116422122
  (0, 8992)	0.34407386324153105
  (0, 3638)	0.34407386324153105
  (1, 1950)	0.15406469307458254
  (1, 2272)	0.38171045858095665
  (1, 16893)	0.30574273067975594
  (1, 1549)	0.2973812219261365
  (1, 2848)	0.19655678171576924
  (1, 6947)	0.19569131906386433
  (1, 5602)	0.7089244615321297
  (1, 3605)	0.26295139224944014
  (2, 5475)	0.4118222937368373
  (2, 6083)	0.3676200487762488
  (2, 9681)	0.5173836673945003
  (2, 15709)	0.443161566926536
  :	:
  (20797, 1347)	0.3321160455506033
  (20797, 13218)	0.25304453797135923
  (20797, 12456)	0.2747303058280698
  (20797, 15047)	0.30856153723375124
  (20797, 12253)	0.24630753514502662
  (20797, 9583)	0.292

In [21]:
def fun(x):
  if x==1:
    return 1
  else:
    return -1

In [23]:
Y_train=Y_train.apply(fun) # converting to 1 and -1 for SVM

In [28]:
model=SVC()
model.fit(X_train,Y_train)

SVC()

In [29]:
y_pred=model.predict(X_train)
accuracy=accuracy_score(y_pred,Y_train)
print('Accuracy on training Data',accuracy)

Accuracy on training Data 0.9983653846153846


In [33]:
def fun1(x):# to convert back to 1 and 0 to submit in file
    if x==1:
        return 1
    else:
        return 0

In [34]:
df_submit['label']=model.predict(X_test)
df_submit['label']=df_submit['label'].apply(fun1).astype(int)
df_submit.to_csv('submission.csv',index=False)